# SysML v2 Python Client - Basic Usage Example

This notebook demonstrates basic usage of the `sysmlv2_client` library to interact with a running SysML v2 API server (like the local Flexo instance).

## 1. Setup

Make sure:
1. The local Flexo Docker containers are running (`docker compose up -d` in `flexo-setup/docker-compose/`).
2. You have performed the initial Postman org setup if required.
3. You have the correct bearer token.
4. The `sysmlv2_client` package is accessible (e.g., by running this notebook from the project root directory or installing the package).

In [37]:
# Adjust the path if necessary to import from the src directory
import sys
import os
import uuid # To generate unique IDs if needed by API for creation
sys.path.insert(0, os.path.abspath('../src'))

from sysmlv2_client import SysMLV2Client, SysMLV2Error, SysMLV2NotFoundError
import json # For pretty printing
from pprint import pprint

## 2. Initialize the Client

Replace `"Bearer YOUR_TOKEN_HERE"` with the actual bearer token obtained from `flexo-setup/docker-compose/env/flexo-sysmlv2.env`.

In [38]:
BASE_URL = "http://localhost:8083" # Flexo SysMLv2 service runs on 8083
# !!! IMPORTANT: Replace with your actual token !!!
BEARER_TOKEN = "Inser Bearer Token Here"

client = None
try:
    client = SysMLV2Client(base_url=BASE_URL, bearer_token=BEARER_TOKEN)
    print("Client initialized successfully!")
except ValueError as e:
    print(f"Error initializing client: {e}")
except Exception as e:
    print(f"An unexpected error occurred during initialization: {e}")

Client initialized successfully!


## 3. Project Operations

In [39]:
if client:
    try:
        print("--- Getting Projects ---")
        projects = client.get_projects()
        print(f"Found {len(projects)} projects.")
        # Print only name and ID for clarity
        if projects:
            for project in projects:
                proj_id = project.get('@id', 'N/A')
                proj_name = project.get('name', 'N/A')
                print(f"  - Name: {proj_name}, ID: {proj_id}")
        else:
            print("  (No projects found)")
    except SysMLV2Error as e:
        print(f"Error getting projects: {e}")

--- Getting Projects ---
Found 9 projects.
  - Name: My Example Project, ID: 2988b76c-0e48-42a1-9f2b-c552874ba6a5
  - Name: My Example Project, ID: 911d0df1-25ef-4faf-9d2a-cbda280452dd
  - Name: My Client Example Project, ID: 02b82e41-66a7-4498-8a38-6dacffedac1c
  - Name: My Example Project, ID: f0be039c-fecc-495c-acfc-0f9011749738
  - Name: My Client Example Project, ID: 462374e8-1b70-4680-b021-cf0efb30ab4d
  - Name: My Example Project, ID: b26987c9-40fb-4ed6-acf7-74a619dbe9b7
  - Name: My Client Example Project, ID: 11702ec3-702a-44f2-8cc6-8438a93cd2c7
  - Name: My Example Project, ID: 1b5d04dd-48af-478c-b955-1efc42ea3e22
  - Name: My Example Project, ID: 0af94881-b7cc-4fe6-accb-524ef3bc24a0


In [40]:
# Create a new project (adjust data as needed)
# This is done to ensure the rest of the notebook has a project ID to work with.
# See the markdown cell below on how to use an existing project instead.
created_project = None
example_project_id = None # Initialize here to ensure it exists

if client:
    new_project_data = {
        "@type": "Project",
        "name": "My Client Example Project",
        "description": "A project created via the Python client for testing"
    }
    try:
        print("\n--- Creating Project ---")
        created_project = client.create_project(new_project_data)
        print("Project created successfully:")
        pprint(created_project)
        # Store the ID for later use
        example_project_id = created_project.get('@id')
        if not example_project_id:
             print("\n*** WARNING: Could not extract project ID ('@id') from response! Subsequent steps might fail. ***")
    except SysMLV2Error as e:
        print(f"Error creating project: {e}")
else:
    print("Client not initialized, skipping project creation.")


--- Creating Project ---
Project created successfully:
{'@id': '39455329-580c-448d-82f8-da06abdec5ef',
 '@type': 'Project',
 'created': '2025-04-08T14:54:46.319279341Z',
 'defaultBranch': {'@id': 'b419d432-db8b-4390-9af7-9ed684cc2c5c'},
 'description': 'A project created via the Python client for testing',
 'name': 'My Client Example Project'}


## 4. Commit and Element Operations

Element creation, update, and deletion are performed by creating new commits with specific payloads in the `change` array.

In [41]:
# --- Commit 1: Create initial elements --- 
commit1_id = None
element1_id_placeholder = str(uuid.uuid4()) # Generate placeholder ID if API doesn't assign one on create
element2_id_placeholder = str(uuid.uuid4())

if client and example_project_id:
    commit1_data = {
        "@type": "Commit",
        "description": "Commit 1: Create initial elements",
        # "previousCommit": None, # First commit might not need this
        "change": [
            {
                "@type": "DataVersion",
                "payload": {
                    "@id": element1_id_placeholder, # May or may not be needed depending on API
                    "@type": "PartDefinition", 
                    "name": "Spacecraft System"
                }
            },
            {
                "@type": "DataVersion",
                "payload": {
                    "@id": element2_id_placeholder,
                    "@type": "PartDefinition",
                    "name": "Payload System"
                }
            }
        ]
    }
    
    try:
        print("\n--- Creating Commit 1 (with element creation) ---")
        commit1_response = client.create_commit(example_project_id, commit1_data)
        print("Commit 1 created successfully:")
        pprint(commit1_response)
        commit1_id = commit1_response.get('@id')
        if not commit1_id:
            print("\n*** WARNING: Could not extract commit ID ('@id') from response! ***")
    except SysMLV2Error as e:
        print(f"Error creating commit 1: {e}")
else:
    print("\nSkipping Commit 1 because client or project ID is missing.")


--- Creating Commit 1 (with element creation) ---
Commit 1 created successfully:
{'@id': '5827f7c1-6841-4243-ad78-38047f89b477',
 '@type': 'Commit',
 'created': '2025-04-08T14:55:04.543675335Z',
 'description': '',
 'owningProject': {'@id': '39455329-580c-448d-82f8-da06abdec5ef'},
 'previousCommit': None}


In [42]:
# --- List elements after Commit 1 to find actual IDs --- 
payload_system_element_id = None
spacecraft_system_element_id = None

if client and example_project_id and commit1_id:
    try:
        print(f"\n--- Listing elements at Commit 1 ({commit1_id}) ---")
        elements_c1 = client.list_elements(example_project_id, commit1_id)
        print(f"Found {len(elements_c1)} elements:")
        pprint(elements_c1)
        
        # Find the IDs based on name (adjust if names aren't unique or stable)
        for elem in elements_c1:
            if elem.get('name') == "Payload System":
                payload_system_element_id = elem.get('@id')
            elif elem.get('name') == "Spacecraft System":
                spacecraft_system_element_id = elem.get('@id')
                
        print(f"\nFound Payload System ID: {payload_system_element_id}")
        print(f"Found Spacecraft System ID: {spacecraft_system_element_id}")
        
        if not payload_system_element_id or not spacecraft_system_element_id:
            print("\n*** WARNING: Could not find expected element IDs after commit 1! ***")
            
    except SysMLV2Error as e:
        print(f"Error listing elements after commit 1: {e}")
else:
    print("\nSkipping element listing because client, project ID, or commit 1 ID is missing.")


--- Listing elements at Commit 1 (5827f7c1-6841-4243-ad78-38047f89b477) ---
Found 2 elements:
[{'@id': '20d06bda-a547-4a90-8023-e7c4cb0ffb92',
  '@type': 'PartDefinition',
  'name': 'Payload System'},
 {'@id': 'b91022ba-41d0-4448-a2b1-538eb8668104',
  '@type': 'PartDefinition',
  'name': 'Spacecraft System'}]

Found Payload System ID: 20d06bda-a547-4a90-8023-e7c4cb0ffb92
Found Spacecraft System ID: b91022ba-41d0-4448-a2b1-538eb8668104


In [43]:
# --- Commit 2: Update Payload System name --- 
commit2_id = None

if client and example_project_id and commit1_id and payload_system_element_id:
    commit2_data = {
        "@type": "Commit",
        "description": "Commit 2: Update Payload System name",
        "previousCommit": {"@id": commit1_id},
        "change": [
            {
                "@type": "DataVersion",
                "identity": {"@id": payload_system_element_id},
                "payload": {
                    "@id": payload_system_element_id, # ID must be included in payload for update
                    "@type": "PartDefinition", 
                    "name": "Updated Payload System Name" # The changed value
                    # Include other existing fields if the API requires the full object on update
                }
            }
        ]
    }
    
    try:
        print("\n--- Creating Commit 2 (with element update) ---")
        commit2_response = client.create_commit(example_project_id, commit2_data)
        print("Commit 2 created successfully:")
        pprint(commit2_response)
        commit2_id = commit2_response.get('@id')
        if not commit2_id:
            print("\n*** WARNING: Could not extract commit ID ('@id') from response! ***")
    except SysMLV2Error as e:
        print(f"Error creating commit 2: {e}")
else:
    print("\nSkipping Commit 2 because prerequisite data is missing.")


--- Creating Commit 2 (with element update) ---
Commit 2 created successfully:
{'@id': '5a5aef4b-1284-4cef-bd73-dd2874e87d09',
 '@type': 'Commit',
 'created': '2025-04-08T14:55:16.851795133Z',
 'description': '',
 'owningProject': {'@id': '39455329-580c-448d-82f8-da06abdec5ef'},
 'previousCommit': None}


In [44]:
# --- List elements after Commit 2 --- 
if client and example_project_id and commit2_id:
    try:
        print(f"\n--- Listing elements at Commit 2 ({commit2_id}) ---")
        elements_c2 = client.list_elements(example_project_id, commit2_id)
        print(f"Found {len(elements_c2)} elements:")
        pprint(elements_c2)
        # You should see 'Updated Payload System Name' here
    except SysMLV2Error as e:
        print(f"Error listing elements after commit 2: {e}")
else:
    print("\nSkipping element listing because client, project ID, or commit 2 ID is missing.")


--- Listing elements at Commit 2 (5a5aef4b-1284-4cef-bd73-dd2874e87d09) ---
Found 2 elements:
[{'@id': '20d06bda-a547-4a90-8023-e7c4cb0ffb92',
  '@type': 'PartDefinition',
  'name': 'Updated Payload System Name'},
 {'@id': 'b91022ba-41d0-4448-a2b1-538eb8668104',
  '@type': 'PartDefinition',
  'name': 'Spacecraft System'}]


In [45]:
# --- Commit 3: Delete Payload System --- 
commit3_id = None

if client and example_project_id and commit2_id and payload_system_element_id:
    commit3_data = {
        "@type": "Commit",
        "description": "Commit 3: Delete Payload System",
        "previousCommit": {"@id": commit2_id},
        "change": [
            {
                "@type": "DataVersion",
                "identity": {"@id": payload_system_element_id},
                "payload": None # Set payload to None for deletion
            }
        ]
    }
    
    try:
        print("\n--- Creating Commit 3 (with element deletion) ---")
        commit3_response = client.create_commit(example_project_id, commit3_data)
        print("Commit 3 created successfully:")
        pprint(commit3_response)
        commit3_id = commit3_response.get('@id')
        if not commit3_id:
            print("\n*** WARNING: Could not extract commit ID ('@id') from response! ***")
    except SysMLV2Error as e:
        print(f"Error creating commit 3: {e}")
else:
    print("\nSkipping Commit 3 because prerequisite data is missing.")


--- Creating Commit 3 (with element deletion) ---
Commit 3 created successfully:
{'@id': 'a9b323fc-b184-465b-a7a9-ff3a1ca1d7ae',
 '@type': 'Commit',
 'created': '2025-04-08T14:55:24.097265053Z',
 'description': '',
 'owningProject': {'@id': '39455329-580c-448d-82f8-da06abdec5ef'},
 'previousCommit': None}


In [46]:
# --- List elements after Commit 3 --- 
if client and example_project_id and commit3_id:
    try:
        print(f"\n--- Listing elements at Commit 3 ({commit3_id}) ---")
        elements_c3 = client.list_elements(example_project_id, commit3_id)
        print(f"Found {len(elements_c3)} elements:")
        pprint(elements_c3)
        # You should only see 'Spacecraft System' here
    except SysMLV2Error as e:
        print(f"Error listing elements after commit 3: {e}")
else:
    print("\nSkipping element listing because client, project ID, or commit 3 ID is missing.")


--- Listing elements at Commit 3 (a9b323fc-b184-465b-a7a9-ff3a1ca1d7ae) ---
Found 1 elements:
[{'@id': 'b91022ba-41d0-4448-a2b1-538eb8668104',
  '@type': 'PartDefinition',
  'name': 'Spacecraft System'}]


## 5. Branch Management

In [47]:
# --- List Branches --- 
if client and example_project_id:
    try:
        print(f"\n--- Listing branches for project {example_project_id} ---")
        branches = client.list_branches(example_project_id)
        print(f"Found {len(branches)} branches:")
        pprint(branches)
    except SysMLV2Error as e:
        print(f"Error listing branches: {e}")
else:
    print("\nSkipping branch listing because client or project ID is missing.")


--- Listing branches for project 39455329-580c-448d-82f8-da06abdec5ef ---
Found 1 branches:
[{'@id': 'b419d432-db8b-4390-9af7-9ed684cc2c5c',
  '@type': 'Branch',
  'created': '2025-04-08T14:54:54.001899136Z',
  'head': {'@id': 'a9b323fc-b184-465b-a7a9-ff3a1ca1d7ae'},
  'name': 'Initial',
  'owningProject': {'@id': '39455329-580c-448d-82f8-da06abdec5ef'},
  'referencedCommit': {'@id': 'a9b323fc-b184-465b-a7a9-ff3a1ca1d7ae'}}]


In [48]:
# --- Create Branch --- 
example_branch_id = None
created_branch = None

# Create branch pointing to the latest commit (commit3)
if client and example_project_id and commit3_id:
    branch_data = {
        "@type": "Branch",
        "name": "feature/my-new-feature",
        "head": {"@id": commit3_id}
    }
    try:
        print(f"\n--- Creating branch 'feature/my-new-feature' pointing to commit {commit3_id} ---")
        created_branch = client.create_branch(example_project_id, branch_data)
        print("Branch created successfully:")
        pprint(created_branch)
        example_branch_id = created_branch.get('@id')
        if not example_branch_id:
            print("\n*** WARNING: Could not extract branch ID ('@id') from response! ***")
    except SysMLV2Error as e:
        print(f"Error creating branch: {e}")
else:
    print("\nSkipping branch creation because prerequisite data is missing.")


--- Creating branch 'feature/my-new-feature' pointing to commit a9b323fc-b184-465b-a7a9-ff3a1ca1d7ae ---
Branch created successfully:
{'@id': '3c74dd55-3082-4209-ba0a-0ac86b47cff6',
 '@type': 'Branch',
 'created': '2025-04-08T14:55:32.131896376Z',
 'head': {'@id': 'a9b323fc-b184-465b-a7a9-ff3a1ca1d7ae'},
 'name': 'feature/my-new-feature',
 'owningProject': {'@id': '39455329-580c-448d-82f8-da06abdec5ef'},
 'referencedCommit': {'@id': 'a9b323fc-b184-465b-a7a9-ff3a1ca1d7ae'}}


In [49]:
# --- Get Branch By ID --- 
if client and example_project_id and example_branch_id:
    try:
        print(f"\n--- Getting branch by ID {example_branch_id} ---")
        branch = client.get_branch_by_id(example_project_id, example_branch_id)
        print("Branch retrieved successfully:")
        pprint(branch)
    except SysMLV2NotFoundError:
        print(f"Branch {example_branch_id} not found.")
    except SysMLV2Error as e:
        print(f"Error getting branch: {e}")
else:
    print("\nSkipping get branch by ID because prerequisite data is missing.")


--- Getting branch by ID 3c74dd55-3082-4209-ba0a-0ac86b47cff6 ---
Branch retrieved successfully:
{'@id': '3c74dd55-3082-4209-ba0a-0ac86b47cff6',
 '@type': 'Branch',
 'created': '2025-04-08T14:55:32.131896376Z',
 'head': {'@id': 'a9b323fc-b184-465b-a7a9-ff3a1ca1d7ae'},
 'name': 'feature/my-new-feature',
 'owningProject': {'@id': '39455329-580c-448d-82f8-da06abdec5ef'},
 'referencedCommit': {'@id': 'a9b323fc-b184-465b-a7a9-ff3a1ca1d7ae'}}


## 6. Tag Management

In [50]:
# --- List Tags --- 
if client and example_project_id:
    try:
        print(f"\n--- Listing tags for project {example_project_id} ---")
        tags = client.list_tags(example_project_id)
        print(f"Found {len(tags)} tags:")
        pprint(tags)
    except SysMLV2Error as e:
        print(f"Error listing tags: {e}")
else:
    print("\nSkipping tag listing because client or project ID is missing.")


--- Listing tags for project 39455329-580c-448d-82f8-da06abdec5ef ---
Found 0 tags:
[]


In [51]:
# --- Create Tag --- 
example_tag_id = None
created_tag = None

# Create tag pointing to commit 2
if client and example_project_id and commit2_id:
    tag_data = {
        "@type": "Tag",
        "name": "v1.0-alpha",
        "taggedCommit": {"@id": commit2_id}
    }
    try:
        print(f"\n--- Creating tag 'v1.0-alpha' pointing to commit {commit2_id} ---")
        created_tag = client.create_tag(example_project_id, tag_data)
        print("Tag created successfully:")
        pprint(created_tag)
        example_tag_id = created_tag.get('@id')
        if not example_tag_id:
            print("\n*** WARNING: Could not extract tag ID ('@id') from response! ***")
    except SysMLV2Error as e:
        print(f"Error creating tag: {e}")
else:
    print("\nSkipping tag creation because prerequisite data is missing.")


--- Creating tag 'v1.0-alpha' pointing to commit 5a5aef4b-1284-4cef-bd73-dd2874e87d09 ---
Tag created successfully:
{'@id': '3fd9d8c7-15ff-4124-be3f-ad73b3f102df',
 '@type': 'Tag',
 'created': '2025-04-08T14:55:43.528545214Z',
 'name': 'v1.0-alpha',
 'owningProject': {'@id': '39455329-580c-448d-82f8-da06abdec5ef'},
 'referencedCommit': {'@id': '5a5aef4b-1284-4cef-bd73-dd2874e87d09'},
 'taggedCommit': {'@id': '5a5aef4b-1284-4cef-bd73-dd2874e87d09'}}


In [52]:
# --- Get Tag By ID --- 
if client and example_project_id and example_tag_id:
    try:
        print(f"\n--- Getting tag by ID {example_tag_id} ---")
        tag = client.get_tag_by_id(example_project_id, example_tag_id)
        print("Tag retrieved successfully:")
        pprint(tag)
    except SysMLV2NotFoundError:
        print(f"Tag {example_tag_id} not found.")
    except SysMLV2Error as e:
        print(f"Error getting tag: {e}")
else:
    print("\nSkipping get tag by ID because prerequisite data is missing.")


--- Getting tag by ID 3fd9d8c7-15ff-4124-be3f-ad73b3f102df ---
Tag retrieved successfully:
{'@id': '3fd9d8c7-15ff-4124-be3f-ad73b3f102df',
 '@type': 'Tag',
 'created': '2025-04-08T14:55:43.528545214Z',
 'name': 'v1.0-alpha',
 'owningProject': {'@id': '39455329-580c-448d-82f8-da06abdec5ef'},
 'referencedCommit': {'@id': '5a5aef4b-1284-4cef-bd73-dd2874e87d09'},
 'taggedCommit': {'@id': '5a5aef4b-1284-4cef-bd73-dd2874e87d09'}}


## 7. Cleanup (Optional)

In [ ]:
# --- Delete Tag --- 
if client and example_project_id and example_tag_id:
    try:
        print(f"\n--- Deleting tag {example_tag_id} ---")
        client.delete_tag(example_project_id, example_tag_id)
        print(f"Tag {example_tag_id} deleted successfully.")
    except SysMLV2NotFoundError:
        print(f"Tag {example_tag_id} not found for deletion.")
    except SysMLV2Error as e:
        print(f"Error deleting tag: {e}")
else:
    print("\nSkipping delete tag because prerequisite data is missing.")

In [ ]:
# --- Delete Branch --- 
if client and example_project_id and example_branch_id:
    try:
        print(f"\n--- Deleting branch {example_branch_id} ---")
        client.delete_branch(example_project_id, example_branch_id)
        print(f"Branch {example_branch_id} deleted successfully.")
    except SysMLV2NotFoundError:
        print(f"Branch {example_branch_id} not found for deletion.")
    except SysMLV2Error as e:
        print(f"Error deleting branch: {e}")
else:
    print("\nSkipping delete branch because prerequisite data is missing.")